In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

## Import framework main libs

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic


## Import strategy from module

In [ ]:
from tmqralphas.strategy_dsp_bandpass import Strategy_DSP_BandPass

## How to port old strategy code to the new framework

[Strategy_DSP_BandPass changes on the GitHub (difference)](https://github.com/trendmanagement/Tmqr-framework-2/commit/5ea55ae6c278d2d3d5d467df8a553b2139c97bc9#diff-090ba08782479b6b92ffe93d6fe6e167)

[New code](https://github.com/trendmanagement/Tmqr-framework-2/blob/5ea55ae6c278d2d3d5d467df8a553b2139c97bc9/tmqralphas/strategy_dsp_bandpass.py)

[Old code](https://github.com/trendmanagement/tmqrexo_alexveden/blob/new_mongo/strategies/strategy_dsp_bandpass.py)

### How to convert old opt_params to the new format

#### Old version
    'opt_params': [
                            OptParamArray('Direction', [-1]),
                            OptParamArray('Filter order',  [3]),
                            OptParamArray('Filter start freq', [0.8878, 0.8881, 0.8889]), # 0 > f < 1
                            OptParamArray('Filter stop freq', [0.7166, 0.7131, 0.6335]), # 0 > f < 1
                            OptParamArray('Filter sigma', [0.8111, 0.8022, 0.8127]),
                            OptParamArray('Rule index', [0]),

                ],
    
#### New version
**Just copy/paste** text after OptParamArray, and keep the params order!

    'wfo_opt_params': [
        ('direction', [-1]),
        ('Filter order',  [3]),
        ('Filter start freq', [0.8878, 0.8881, 0.8889]), # 0 > f < 1
        ('Filter stop freq', [0.7166, 0.7131, 0.6335]), # 0 > f < 1
        ('Filter sigma', [0.8111, 0.8022, 0.8127]),
        ('Rule index', [0]),
    ]

In [ ]:
ALPHA_CONTEXT = {
        'name': '6C_NewFramework_DSP_BandPass', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.6C_ContFutEOD',      # Name of EXO index to trade
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 2,  # Number of months is OOS period
            'iis_periods': 12,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerGenetic, 
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 5,
            'nbest_fitness_method': 'max',
            'population_size': 50, 
            'number_generations': 30, 
            # 'rand_seed': 1, # Uncomment this parameter to make genetic results repeatable
        },
        'wfo_opt_params': [
            ('direction', [1]),
            ('Filter order',  [3]),
            ('Filter start freq', [0.8878, 0.8881, 0.8889]), # 0 > f < 1
            ('Filter stop freq', [0.7166, 0.7131, 0.6335]), # 0 > f < 1
            ('Filter sigma', [0.8111, 0.8022, 0.8127]),
            ('Rule index', [0]),
        ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'netprofit'
    }

# Run the alpha

In [ ]:
# DataManager is a core class of the framework
dm = DataManager()

# Init alpha class and run
alpha = Strategy_DSP_BandPass(dm, **ALPHA_CONTEXT)    

In [ ]:
alpha.run()

# Equity

In [ ]:
alpha.stats['series']['equity'].plot()